In [2]:
import time
import psutil

def get_system_usage():
    process = psutil.Process()
    cpu_usage = process.cpu_percent(interval=1)
    memory_info = process.memory_info()
    memory_usage = memory_info.rss / (1024 ** 2)
    return cpu_usage, memory_usage

# Tiempo
start_time = time.time()
start_cpu_usage, start_memory_usage = get_system_usage()

In [3]:
!pwd

/data/home/djcatalan/Documents/BIRADS/RAG


In [4]:
#!xdg-open .

In [5]:
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core.indices.service_context import ServiceContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

cache_folder = "./model_cache"
#embed_model = HuggingFaceEmbedding(
#    model_name="BAAI/bge-small-en", cache_folder = cache_folder)
embed_model = resolve_embed_model("local:BAAI/bge-small-en")
#llm = 'local:mistral-7b-instruct-v0.1.Q4_K_M.gguf'
#llm = 'local:mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf'
#llm = 'local:gemma-7b-it.fp16.gguf'

llm='local:mistral-7b-instruct-v0.1.Q4_K_M.gguf'
Settings.llm=llm
Settings.embed_model=embed_model

No sentence-transformers model found with name BAAI/bge-small-en. Creating a new one with MEAN pooling.
/data/home/djcatalan/.conda/envs/new_djcatalan/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader:

In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import re

# Directorio
path= "/data/home/djcatalan/Documents/BIRADS/RAG/Mamografies.csv"

# Leer .csv
df = pd.read_csv(path, encoding='ISO-8859-1', delimiter=';')

# Normalizar texto html
df['Contingut'] = df['Contingut'].apply(lambda x: BeautifulSoup(str(x).lower(), "html.parser").get_text() if isinstance(x, str) and '<' in x and '>' in x else x)

# Normalizar prueba diagnostica
df['Prova'] = df['Prova'].apply(lambda x: re.sub(r'\d+\.?', '', str(x)).replace('[', '').replace(']', '').strip().lower() if isinstance(x, str) else str(x).lower())

# Eliminar datos personales
def remove_data(text):
    if isinstance(text, str):
        return re.sub('barcelona.*', 'barcelona', text)
    return text

df['Contingut']=df['Contingut'].apply(remove_data)

# Columnas son numéricas
df['Historia'] = pd.to_numeric(df['Historia'], errors='coerce')
df['Exploracio'] = pd.to_numeric(df['Exploracio'], errors='coerce')


In [7]:
# Verificar si hay duplicados
duplicados = df.duplicated(subset=["Historia", "Exploracio"], keep=False)

# Eliminar duplicados
df_dedup = df.drop_duplicates(subset=["Historia", "Exploracio"])

# Comparar tamaños antes y después
df_before = len(df)
df_after = len(df_dedup)

print(f'Antes: {df_before}')
print(f'Después: {df_after}')

Antes: 18708
Después: 17164


In [8]:
print(df.head())

   Exploracio   Historia        Data       Prova  \
0     3721809   109368.0  03/01/2018  mamografia   
1     3540069   193146.0  03/01/2018  mamografia   
2     3543745  1538814.0  03/01/2018  mamografia   
3     3540030   213314.0  03/01/2018  mamografia   
4     3537201  1250727.0  03/01/2018  mamografia   

                                           Contingut  
0   mamografía.motivo de exploración: control gin...  
1  mamografía.motivo de exploración: control onco...  
2  ecografía + mamografía bilateralmotivo de soli...  
3  mamografía.motivo de exploración: control onco...  
4   mamografía.motivo de exploración: control onc...  


In [9]:
from llama_index.core import Document
from llama_index.core.schema import MetadataMode

if not isinstance(df_dedup, pd.DataFrame):
    raise ValueError ("no Dataframe")

# Crear Document
documents = []

for idx, row in df_dedup.iterrows():
    document_id = row["Exploracio"]

    metadata = {
        "Numero exploracion": row["Exploracio"],
        "Numero historia": row["Historia"],
        "Fecha de consulta": row["Data"],
        "Prueba diagnostica": row["Prova"],
    }
    document = Document(
        text=row["Contingut"],
        id_=document_id,
        metadata=metadata,
        excluded_llm_metadata_keys=['file_path', 'file_type', 'file_size', 'creation_date', 'last_modified_date'],
        metadata_separator="::",
        metadata_template="{key}=>{value}",
        text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
    )
    
    documents.append(document)

print(f"Total: {len(documents)}")
    

Total: 17164


In [10]:
for i, doc in enumerate(documents[:2]):
    print(f"Document {i} ID: {doc.id_}")
    print("The LLM see this: \n", doc.get_content(metadata_mode=MetadataMode.LLM))
    #print("The embedding model see this: \n", doc.get_content(metadata_mode=MetadataMode.EMBED))

Document 0 ID: 3721809
The LLM see this: 
 Metadata: Numero exploracion=>3721809
Numero historia=>109368.0
Fecha de consulta=>03/01/2018
Prueba diagnostica=>mamografia
-----
Content:  mamografía.motivo de exploración: control ginecológico.resultados:mamas simétricas con predominio del tejido graso.no se observan lesiones focales sospechosas. conclusión: negativo. birads 1.densidad mamaria: a.
Document 1 ID: 3540069
The LLM see this: 
 Metadata: Numero exploracion=>3540069
Numero historia=>193146.0
Fecha de consulta=>03/01/2018
Prueba diagnostica=>mamografia
-----
Content: mamografía.motivo de exploración: control oncológico. antecedente de tumorectomía bilateral.resultados:mamas con tejido fibroepitelial denso de forma heterogénea. se observan cambios secundarios a tratamiento quirúrgico conservador y radioterapia de ambas mamas. la cicatriz de mi es un tanto densa, pero sin cambios significativos respecto a controles previos y en la cicatriz de md se observan calcificaciones de lipone

In [11]:

# Definir apartados
def extract_conclusion(text):
    conclusion_pattern = r'\b(?:conclusi[óo]ne?s?|impresi[óo]ne?s?(\s+diagn[oó]stica)?)\s*:\s*(.*)'
    match = re.search(conclusion_pattern, text, re.DOTALL)
    if match:
        return match.group(2).strip()
    return ""


In [12]:
import re

# Función para limpiar el texto de caracteres especiales
def clean_text(text):
    # Definir la expresión regular para permitir letras, números, espacios, puntuación y tildes
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s.,;:!?áéíóúÁÉÍÓÚñÑ]', '', text)
    return cleaned_text

# Definir función para extraer conclusiones
def extract_conclusion(text):
    text = clean_text(text)  # Limpiar el texto antes de extraer la conclusión
    conclusion_pattern = r'\b(?:conclusi[oó]ne?s?|impresi[oó]ne?s?(\s+diagn[oó]stica)?)\s*:\s*(.*)'
    match = re.search(conclusion_pattern, text, re.DOTALL)
    if match:
        return match.group(2).strip()
    return ""

# Extraer conclusiones y almacenar en una lista
conclusions = [extract_conclusion(doc.text) for doc in documents]

# Crear un DataFrame a partir de las conclusiones
df_conclusions = pd.DataFrame(conclusions, columns=['Conclusion'])

# Verificar y contar los duplicados
duplicated_conclusions = df_conclusions.duplicated(keep=False)
duplicated_df = df_conclusions[duplicated_conclusions]
duplicated_count = duplicated_df['Conclusion'].value_counts()

print("Conclusiones duplicadas y su conteo:")
print(duplicated_count)

Conclusiones duplicadas y su conteo:
Conclusion
                                                                                                                                                                                              2115
negativo. birads 1.densidad mamaria: b.                                                                                                                                                        543
negativo. birads 1.densidad mamaria: c.                                                                                                                                                        449
lesiones benignas. birads 2.densidad mamaria: b.                                                                                                                                               280
lesiones benignas. birads 2.densidad mamaria: c.                                                                                                                            

In [13]:
# Eliminar duplicados
df_conclusions_unique = df_conclusions.drop_duplicates()

# Contar conclusiones antes y después de eliminar duplicados
conclusions_before = len(df_conclusions)
conclusions_after = len(df_conclusions_unique)

print(f"Total de conclusiones antes de eliminar duplicados: {conclusions_before}")
print(f"Total de conclusiones después de eliminar duplicados: {conclusions_after}")

Total de conclusiones antes de eliminar duplicados: 17164
Total de conclusiones después de eliminar duplicados: 7296


In [14]:
# Extraccion de BIRADS en Conclusion
def extract_bi_rads(text):
    bi_rads_matches = re.findall(r'bi\s*-?\s*rads\s*:? ?(4\s*[a-c]?|6|5|3|2|1|0)', text)
    if not bi_rads_matches:
        return 'no especifica BIRADS en el documento'
    
    bi_rads_norm = set()
    for match in bi_rads_matches:
        match = match.replace(" ","")
        if match.startswith('4'):
            if len(match) > 1:
                match = '4' + match[1].lower()
        bi_rads_norm.add(match)
    bi_rads = ", ".join(f"BIRADS {item}" for item in sorted(bi_rads_norm)) # Concatena
    return bi_rads

# Extraer lateralidad en Conclusion
def extract_lateralidad(text):
    lateralidad_matches = r'\b(derecha|izquierda|esquerra|dreta|md|mi)\b'
    matches = re.findall(lateralidad_matches, text)
    if not matches:
        return "no especifica lateralidad de la mama"
    
    lateralidad_norm = set()
    for match in matches:
        if match in ['derecha', 'dreta', 'md']:
            lateralidad_norm.add('mama derecha')
            text = re.sub(r'\b' + re.escape(match) + r'\b', "mama derecha", text)
        elif match in ['izquierda', 'esquerra', 'mi']:
            lateralidad_norm.add('mama izquierda')
            text = re.sub(r'\b' + re.escape(match) + r'\b', "mama izquierda", text)
        
    return ", ".join(sorted(lateralidad_norm)) if lateralidad_norm else "no especifica lateralidad de la mama"

In [15]:
# Aplicar funciones extract_bi_rads y extract_lateralidad a las conclusiones únicas
bi_rads_results = df_conclusions_unique['Conclusion'].apply(extract_bi_rads)
lateralidad_results = df_conclusions_unique['Conclusion'].apply(extract_lateralidad)

# Crear un diccionario para mapear conclusiones a los resultados de las funciones
bi_rads_dict = dict(zip(df_conclusions_unique['Conclusion'], bi_rads_results))
lateralidad_dict = dict(zip(df_conclusions_unique['Conclusion'], lateralidad_results))

In [16]:
# Actualizar metadatos
for doc, conclusion_text in zip(documents, conclusions):
    doc.metadata["BI-RADS"] = bi_rads_dict[conclusion_text]
    doc.metadata["Lateralidad mama"] = lateralidad_dict[conclusion_text]

for doc in documents[:5]:
    print("The LLM see this: \n", doc.get_content(metadata_mode=MetadataMode.LLM))
    print("The embedding model see this: \n", doc.get_content(metadata_mode=MetadataMode.EMBED))

The LLM see this: 
 Metadata: Numero exploracion=>3721809
Numero historia=>109368.0
Fecha de consulta=>03/01/2018
Prueba diagnostica=>mamografia
BI-RADS=>BIRADS 1
Lateralidad mama=>no especifica lateralidad de la mama
-----
Content:  mamografía.motivo de exploración: control ginecológico.resultados:mamas simétricas con predominio del tejido graso.no se observan lesiones focales sospechosas. conclusión: negativo. birads 1.densidad mamaria: a.
The embedding model see this: 
 Metadata: Numero exploracion=>3721809
Numero historia=>109368.0
Fecha de consulta=>03/01/2018
Prueba diagnostica=>mamografia
BI-RADS=>BIRADS 1
Lateralidad mama=>no especifica lateralidad de la mama
-----
Content:  mamografía.motivo de exploración: control ginecológico.resultados:mamas simétricas con predominio del tejido graso.no se observan lesiones focales sospechosas. conclusión: negativo. birads 1.densidad mamaria: a.
The LLM see this: 
 Metadata: Numero exploracion=>3540069
Numero historia=>193146.0
Fecha de con

In [18]:
birads_values = [doc.metadata["BI-RADS"] for doc in documents if "BI-RADS" in doc.metadata]
birads_counts = pd.Series(birads_values).values_count())

In [ ]:
# TRANSFORMATION
from llama_index.core.node_parser import SentenceSplitter
from typing import List

# Homogenizar BIRADS y Lateralidad
def homogenizer(text):
    text = re.sub(r'\bbi\s*-?\s*rads\s*:? ?', 'birads ', text)
    text = re.sub(r'\b4\s*(a|b|c)\b', r'4\1', text)
    text = re.sub(r'mama\s*derecha|mama\s*dreta|\bmd\b', "mama derecha", text)
    text = re.sub(r'mama\s*izquierda|mama\s*esquerra|\bmi\b', "mama izquierda", text)
    return text

def clean_puntuation(text):
    text = re.sub(r'[^A-Za-z0-9\s]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def eliminate_tildes(text, remove_tildes= True) -> str:
    # Convertir a minúsculas
    text = text.lower()
    if remove_tildes:
        # Reemplazar tildes agudas y graves por vocales base
        text = re.sub('[áà]', 'a', text)
        text = re.sub('[éè]', 'e', text)
        text = re.sub('[íì]', 'i', text)
        text = re.sub('[óò]', 'o', text)
        text = re.sub('[úù]', 'u', text)
    
    # Eliminar espacios adicionales
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Splitting y eliminacion de caracteres
patterns = {
        'Motivo': r'\b(ecografia|mamografia*motivo|mamografia\smotivo|mamografias*ecografia|mamografia\secografia)\b', 
        'Resultados': r'\b(resultado|resultados)\b', 
        'Conclusion': r'\b(conclusion|conclusiones|impresion|impresiones|impresiones+diagnostica)\b'
    }

def custom_sentence_splitter(text, chunk_size=None):
    sentences = []
    combined_pattern = '|'.join(patterns.values())
    matches = list(re.finditer(combined_pattern, text))

    if not matches:
        chunk_size = chunk_size or 500
        for i in range(0, len(text), chunk_size):
            sentences.append(text[i:i+chunk_size].strip())
        return sentences

    prev_end = 0
    for i, match in enumerate(matches):
        start = match.start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        if prev_end < start:
            sentences.append(text[prev_end:start].strip())
        sentences.append(text[start:end].strip())
        prev_end = end
    
    sentences = [s for s in sentences if s]
    return sentences
    
class CustomSentenceSplitter(SentenceSplitter):
    def _split_text(self, text: str, chunk_size: int) -> List[str]:
        """
        Override to use custom sentence.
        """
        return custom_sentence_splitter(text, chunk_size)

In [ ]:
assert all(isinstance(doc, Document) for doc in documents)

In [ ]:
from llama_index.core.storage.docstore import SimpleDocumentStore

# Limpieza
for doc in documents:
    doc.text = homogenizer(doc.text)
    doc.text = eliminate_tildes(doc.text)
    doc.text = clean_puntuation(doc.text)

In [ ]:
for doc in documents[:20]:
    print("The LLM see this: \n", doc.get_content(metadata_mode=MetadataMode.LLM))

In [ ]:
end_time = time.time()
end_cpu_usage, end_memory_usage = get_system_usage()

# Calcular
processing_time = end_time - start_time
cpu_usage_diff = end_cpu_usage - start_cpu_usage
memory_usage_dif = end_memory_usage - start_memory_usage

print(f"Tiempo de procedamiento: {processing_time:.2f} segundos")
print(f"Incremento en el uso de CPU: {cpu_usage_diff:.2f}%")
print(f"Incremento en el uso de memoria: {memory_usage_dif:.2f} MB")

In [ ]:
from llama_index.core.storage.docstore import SimpleDocumentStore
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5", cache_folder = cache_folder)

def get_text_embedding(text):
    with torch.no_grad():
        embedding = embed_model(text.to(device))
    return embedding.cpu().numpy()

for doc in documents:
    embedding = embed_model.get_text_embedding(doc.get_content())
    doc.embedding = embedding

In [ ]:
docstore = SimpleDocumentStore()
docstore.add_documents(documents)

In [ ]:
for doc in documents[:5]:
    print(f"Document: {doc.id_}, Emb: {doc.embedding[:10]}...")

In [ ]:
import numpy as np
from collections import defaultdict

embeddings = [tuple(doc.embedding) for doc in document]

embedding_counts = defaultdict(int)


In [ ]:
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.core.schema import IndexNode
import os
from tqdm.notebook import tqdm

def build_nodes(documents):
    nodes = []
    splitter = CustomSentenceSplitter(chunking_tokenizer_fn=None)

    for idx, doc in enumerate(tqdm(documents)):
        cur_nodes = splitter.get_nodes_from_documents([doc])
        for cur_node in cur_nodes:
            # ID will be base + parent
            file_name = doc.metadata["Numero exploracion"]
            new_node = IndexNode(
                text=cur_node.text or "None",
                index_id=str(file_name),
                metadata=doc.metadata,
            )
            nodes.append(new_node)
            
    print("num nodes: " + str(len(nodes)))
    return nodes

In [ ]:
nodes = build_nodes(documents)

In [ ]:
expected_num_nodes = len(documents)
actual_num_nodes = len(nodes)

print(f"Extected: {expected_num_nodes}")
print(f"Actual: {actual_num_nodes}")

for node in nodes[:5]:
    print(f"Node ID: {node.index_id}, Metadata: {node.metadata}")

In [ ]:
def save_index(nodes, embed_model, out_path):
    out_path = os.path.abspath(out_path)
    #print(f"Ruta {out_path}")

    #if not os.access(out_path, os.W_OK):
        #print("No hay permiso")
        #return None
    
    if not os.path.exists(out_path):
        index = VectorStoreIndex(nodes, embed_model=embed_model)
        index.set_index_id("simple_index")
        index.storage_context.persist(f"./{out_path}")
    else:
        print("Directorio si existe")
        # rebuild storage context
        storage_context = StorageContext.from_defaults(
            persist_dir=out_path
        )
        # load index
        index = load_index_from_storage(
            storage_context, index_id="simple_index", embed_model=embed_model
        )

    return index

In [ ]:
# Guardar indice

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5", cache_folder = cache_folder)

out_path = "./chunck_index"
index = save_index(nodes, embed_model, out_path)

In [ ]:
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.indices.query.embedding_utils import get_top_k_embeddings
from llama_index.core import QueryBundle
from llama_index.core.schema import NodeWithScore
from typing import List, Any, Optional


class HybridRetriever(BaseRetriever):
    """Hybrid retriever."""

    def __init__(
        self,
        vector_index,
        docstore,
        similarity_top_k: int = 2,
        out_top_k: Optional[int] = None,
        alpha: float = 0.5,
        **kwargs: Any,
    ) -> None:
        """Init params."""
        super().__init__(**kwargs)
        self._vector_index = vector_index
        self._embed_model = vector_index._embed_model
        self._retriever = vector_index.as_retriever(
            similarity_top_k=similarity_top_k
        )
        self._out_top_k = out_top_k or similarity_top_k
        self._docstore = docstore
        self._alpha = alpha

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        # first retrieve chunks
        nodes = self._retriever.retrieve(query_bundle.query_str)

        # get documents, and embedding similiaryt between query and documents

        ## get doc embeddings
        docs = [self._docstore.get_document(n.node.index_id) for n in nodes]
        doc_embeddings = [d.embedding for d in docs]
        query_embedding = self._embed_model.get_query_embedding(
            query_bundle.query_str
        )

        ## compute doc similarities
        doc_similarities, doc_idxs = get_top_k_embeddings(
            query_embedding, doc_embeddings
        )

        ## compute final similarity with doc similarities and original node similarity
        result_tups = []
        for doc_idx, doc_similarity in zip(doc_idxs, doc_similarities):
            node = nodes[doc_idx]
            # weight alpha * node similarity + (1-alpha) * doc similarity
            full_similarity = (self._alpha * node.score) + (
                (1 - self._alpha) * doc_similarity
            )
            full_similarity = round(full_similarity, 2)
            print(
                f"Doc {doc_idx} (node score, doc similarity, full similarity): {(round(node.score, 2), round(doc_similarity, 2), full_similarity)}"
            )
            result_tups.append((full_similarity, node))

        result_tups = sorted(result_tups, key=lambda x: x[0], reverse=True)
        
        # update scores
        for full_score, node in result_tups:
            node.score = full_score

        return [n for _, n in result_tups][:out_top_k]

In [ ]:
top_k = 10
out_top_k = 3
hybrid_retriever = HybridRetriever(
    index, docstore, similarity_top_k=top_k, out_top_k=3, alpha=0.5
)

base_retriever = index.as_retriever(similarity_top_k=out_top_k)

In [ ]:
def show_nodes(nodes, out_len: int = 5000):
    for idx, n in enumerate(nodes):
        lateralidad = n.metadata.get('Lateralidad mama')
        bi_rads = n.metadata.get('BI-RADS')
        print(f"\n\n >> ID, Lateralidad: {lateralidad}, BI-RADS: {bi_rads}")
        print(n.get_content()[:out_len])

In [ ]:
query_str = "Muestra numero de informes con birads 5 en la mama izquierda"

In [ ]:
nodes = hybrid_retriever.retrieve(query_str)

In [ ]:
show_nodes(nodes)

In [ ]:
end_time = time.time()
end_cpu_usage, end_memory_usage = get_system_usage()

# Calcular
processing_time = end_time - start_time
cpu_usage_diff = end_cpu_usage - start_cpu_usage
memory_usage_dif = end_memory_usage - start_memory_usage

print(f"Tiempo de procedamiento: {processing_time:.2f} segundos")
print(f"Incremento en el uso de CPU: {cpu_usage_diff:.2f}%")
print(f"Incremento en el uso de memoria: {memory_usage_dif:.2f} MB")

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

# assemble query engine
custom_query_engine = RetrieverQueryEngine(hybrid_retriever)
base_query_engine = index.as_query_engine(similarity_top_k=out_top_k)

In [ ]:
response_str = custom_query_engine.query(query_str)
print(str(response_str))

In [ ]:
questions= ["Muestra numero de informes con birads 0 en la mama izquierda",
            "Muestra numero de informes con birads 0 en la mama derecha"]

responses = {}

for question_str in questions:
    print(f"Pregunta: {question_str}")
    response_str = custom_query_engine.query(question_str)
    responses[question_str] = response_str
    print(f"Respuesta: {str(response_str)}\n")

In [ ]:
questions= ["Muestra numero de informes con birads 1 en la mama izquierda",
            "Muestra numero de informes con birads 1 en la mama derecha"]

responses = {}

for question_str in questions:
    print(f"Pregunta: {question_str}")
    response_str = custom_query_engine.query(question_str)
    responses[question_str] = response_str
    print(f"Respuesta: {str(response_str)}\n")

In [ ]:
questions= ["Muestra numero de informes con birads 2 en la mama izquierda",
            "Muestra numero de informes con birads 2 en la mama derecha"]

responses = {}

for question_str in questions:
    print(f"Pregunta: {question_str}")
    response_str = custom_query_engine.query(question_str)
    responses[question_str] = response_str
    print(f"Respuesta: {str(response_str)}\n")

In [ ]:
questions= ["Muestra numero de informes con birads 3 en la mama izquierda",
            "Muestra numero de informes con birads 3 en la mama derecha"]

responses = {}

for question_str in questions:
    print(f"Pregunta: {question_str}")
    response_str = custom_query_engine.query(question_str)
    responses[question_str] = response_str
    print(f"Respuesta: {str(response_str)}\n")

In [ ]:
questions= ["Muestra numero de informes con birads 4 en la mama izquierda",
            "Muestra numero de informes con birads 4 en la mama derecha"]

responses = {}

for question_str in questions:
    print(f"Pregunta: {question_str}")
    response_str = custom_query_engine.query(question_str)
    responses[question_str] = response_str
    print(f"Respuesta: {str(response_str)}\n")

In [ ]:
questions= ["Muestra numero de informes con birads 4A en la mama izquierda",
            "Muestra numero de informes con birads 4A en la mama derecha"]

responses = {}

for question_str in questions:
    print(f"Pregunta: {question_str}")
    response_str = custom_query_engine.query(question_str)
    responses[question_str] = response_str
    print(f"Respuesta: {str(response_str)}\n")

In [ ]:
questions= ["Muestra numero de informes con birads 4B en la mama izquierda",
            "Muestra numero de informes con birads 4B en la mama derecha"]

responses = {}

for question_str in questions:
    print(f"Pregunta: {question_str}")
    response_str = custom_query_engine.query(question_str)
    responses[question_str] = response_str
    print(f"Respuesta: {str(response_str)}\n")

In [ ]:
questions= ["Muestra numero de informes con birads 4C en la mama izquierda",
            "Muestra numero de informes con birads 4C en la mama derecha"]

responses = {}

for question_str in questions:
    print(f"Pregunta: {question_str}")
    response_str = custom_query_engine.query(question_str)
    responses[question_str] = response_str
    print(f"Respuesta: {str(response_str)}\n")

In [ ]:
questions= ["Muestra numero de informes con birads 5 en la mama izquierda",
            "Muestra numero de informes con birads 5 en la mama derecha"]

responses = {}

for question_str in questions:
    print(f"Pregunta: {question_str}")
    response_str = custom_query_engine.query(question_str)
    responses[question_str] = response_str
    print(f"Respuesta: {str(response_str)}\n")

In [ ]:
questions= ["Muestra numero de informes con birads 6 en la mama izquierda",
            "Muestra numero de informes con birads 6 en la mama derecha"]

responses = {}

for question_str in questions:
    print(f"Pregunta: {question_str}")
    response_str = custom_query_engine.query(question_str)
    responses[question_str] = response_str
    print(f"Respuesta: {str(response_str)}\n")

In [ ]:
questions= ["¿Cuál es el birads para esta frase: lesiones benignas?",
            "¿Cuál es el birads para esta frase: exploracion incompleta, exploracion insuficiente o valoracion adicional?", 
            "¿Cuál es el birads para esta frase: probablemente benignas?",
            "¿Cuál es el birads para esta frase: alta sospecha de malignidad?",
            "¿Cuál es el birads para esta frase: baja sospecha de malignidad?",
            "¿Cuál es el birads para esta frase: moderada sospecha de malignidad?",
            "¿Cuál es el birads para esta frase: altamente sospechosa de malignidad?",
            "¿Cuál es el birads para esta frase: realización de biopsia, realización de marcaje o neoplasias?"]

responses = {}

for question_str in questions:
    print(f"Pregunta: {question_str}")
    response_str = custom_query_engine.query(question_str)
    responses[question_str] = response_str
    print(f"Respuesta: {str(response_str)}\n")

In [ ]:
questions= ["¿Me podas decir cuáles son los tipos de mamas indicados en los informes?"]

responses = {}

for question_str in questions:
    print(f"Pregunta: {question_str}")
    response_str = custom_query_engine.query(question_str)
    responses[question_str] = response_str
    print(f"Respuesta: {str(response_str)}\n")

In [ ]:
questions= ["¿Me podrías decir si hay informes escritos en lengua catalana?"]

responses = {}

for question_str in questions:
    print(f"Pregunta: {question_str}")
    response_str = custom_query_engine.query(question_str)
    responses[question_str] = response_str
    print(f"Respuesta: {str(response_str)}\n")

In [ ]:
questions= ["¿Me podrías decir si hay informes escritos en lengua catalana?"]

responses = {}

for question_str in questions:
    print(f"Pregunta: {question_str}")
    response_str = custom_query_engine.query(question_str)
    responses[question_str] = response_str
    print(f"Respuesta: {str(response_str)}\n")

In [ ]:
base_nodes = base_retriever.retrieve(query_str)

In [ ]:
show_nodes(base_nodes)

In [ ]:
question_str= "¿Hay informes con birads 4C en la mama izquierda? ¿Cúal número de historia?"
response_str = custom_query_engine.query(question_str)
print(str(response_str))

In [ ]:
question_str= "¿Hay informes con birads 4B en la mama izquierda? ¿Cúal número de historia?"
response_str = custom_query_engine.query(question_str)
print(str(response_str))

In [ ]:
question_str= "¿Hay informes con birads 4B en la mama izquierda? ¿Cúal número de historia?"
response_str = custom_query_engine.query(question_str)
print(str(response_str))

In [ ]:
question_str= "¿Hay informes con birads 4C en la mama izquierda? ¿Cúal número de historia?"
response_str = custom_query_engine.query(question_str)
print(str(response_str))

In [ ]:
question_4= "Muestra 3 informes que tengan una categoría birads 4A y afecten la mama izquierda"
response_4 = custom_query_engine.query(question_4)
print(str(response_4))

In [ ]:
response_base_4 = base_query_engine.query(question_4)
print(str(response_base_4))

In [ ]:
question_5= "¿Existen informes con birads 7 en la mama izquierda?"
response_5 = custom_query_engine.query(question_5)
print(str(response_5))

In [ ]:
response_base_5 = base_query_engine.query(question_5)
print(str(response_base_5))

In [ ]:
question_6= "¿Existen informes con birads 4B me podas dar el número de exploracion?"
response_6 = custom_query_engine.query(question_6)
print(str(response_6))

In [ ]:
question_7= "Muestra 3 informes que tengan una categoría birads 4B"
response_7 = custom_query_engine.query(question_7)
print(str(response_7))

In [ ]:
questions = [
    "¿Qué informes tienen una clasificación BI-RADS 5 en la mama izquierda?",
    "Muestra los informes clasificados como BI-RADS 4 en la mama derecha",
    "¿Hay informes con BI-RADS 3 en la mama izquierda?",
    "Muestra 3 informes que tengan una categoría BI-RADS 4A y afecten la mama izquierda",
    "¿Existen informes con BI-RADS 7 en la mama izquierda?"
]

In [ ]:
for q in questions:
    response_all = custom_query_engine.from_args(q)
    print("Question -", q, "\n")
    print("Response -", response_all, "\n\n")

In [ ]:
contexts = hybrid_retriever.retrieve(questions[1])
for c in contexts:
    if "birads" in c.text:
        print("\nSí\n")
    else:
        print("\nNo\n")

print([c.text+"*"*100 for c in contexts])